In [ ]:
from google.colab import drive
drive.mount('/content/drive')
% cd 'drive/My Drive/ece c247/project'
% ls

Mounted at /content/drive
/content/drive/My Drive/ece c247/project
bestM.pt                gen                     X_train_valid.npy
disc                    logs/                   y_test.npy
EEG_loading.ipynb       person_test.npy         y_train_valid.npy
fake_X_train_valid.npy  person_train_valid.npy
fake_y_train_valid.npy  X_test.npy


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from torchvision import transforms, utils
import time
import torch.nn.functional as F
from scipy.signal import savgol_filter


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")

X_train_valid = X_train_valid[:,:,:500]

print('y_train_valid', y_train_valid[:10])

print(np.amax(X_train_valid))
print(np.amin(X_train_valid))
print(np.average(X_train_valid))
print(np.std(X_train_valid))

print(np.percentile(X_train_valid, 80))
print(np.percentile(X_train_valid, 20))

print(np.percentile(X_train_valid, 95))
print(np.percentile(X_train_valid, 5))

print(np.percentile(X_train_valid, 99))
print(np.percentile(X_train_valid, 1))

y_train_valid -= 769

# copy numpy data to tensor
X_train_valid_tensor = torch.from_numpy(X_train_valid).float().to(device)
y_train_valid_tensor = torch.from_numpy(y_train_valid).float().long().to(device) # do not forget .long()


cuda
y_train_valid [771 772 769 769 769 769 771 770 772 772]
90.087890625
-98.828125
0.37977588803594453
10.898771123329505
8.88671875
-8.203125
18.310546875
-17.138671875
27.880859375
-26.171875


In [ ]:
DROPOUT=0.5
class discriminator(nn.Module):
    def __init__(self,classes=4):
        super(discriminator,self).__init__()
        self.conv11 = nn.Conv2d(1, 25, (1, 5))
        self.conv12 = nn.Conv2d(25, 25, (22, 1))
        self.batchnorm1 = nn.BatchNorm2d(25)
        self.pool = nn.MaxPool2d((1,2))
        self.dp = nn.Dropout(p=DROPOUT)
        
        # Layer 2
        self.conv2 = nn.Conv2d(25, 50, (1, 5))
        self.batchnorm2 = nn.BatchNorm2d(50)
        
        # Layer 3

        self.conv3 = nn.Conv2d(50, 100, (1, 5))
        self.batchnorm3 = nn.BatchNorm2d(100)

        # Layer 4

        # self.conv4 = nn.Conv2d(100, 200, (1, 5))
        # self.batchnorm4 = nn.BatchNorm2d(200)

        
        # FC Layer
        self.fc_class = nn.Linear(100 * 59, 4)
        self.fc_source=nn.Linear(100 * 59, 1)

        self.sig=nn.Sigmoid()
        self.soft=nn.Softmax(dim=1)

    def forward(self,x):

        # Layer 1
        # reshape x: (B, 22, 1000) -> (B, 1, 22, 1000), B,C,H,W
        x = x.view(-1, 1, 22, 500)

        x = self.conv12(self.conv11(x))
        x = self.batchnorm1(x)
        x = F.elu(x)
        x = self.pool(x)
        x = self.dp(x)
        
        # Layer 2
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = F.elu(x)
        x = self.pool(x)
        x = self.dp(x)
        
        # Layer 3
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = F.elu(x)
        x = self.pool(x)
        x = self.dp(x)
        
        # Layer 4
        # x = self.conv4(x)
        # x = self.batchnorm4(x)
        # x = F.elu(x)
        # x = self.pool(x)
        # x = self.dp(x)

        # print(x.shape)
        x = x.reshape(-1, 100 * 59)

        rf=self.sig(self.fc_source(x))#checks source of the data---i.e.--data generated(fake) or from training set(real)
        c=self.soft(self.fc_class(x))#checks class(label) of data--i.e. to which label the data belongs in the dataset
        
        return rf,c 

In [ ]:
class generator(nn.Module):

    #generator model
    def __init__(self,in_channels):
        super(generator,self).__init__()
        self.label_emb = nn.Embedding(num_embeddings=4, embedding_dim=in_channels)

        self.fc1=nn.Linear(in_channels,384)
        # 1 -> 8
        self.t1=nn.Sequential(
            nn.ConvTranspose2d(in_channels=384,out_channels=192,kernel_size=(1,8),stride=1,padding=0),
            nn.BatchNorm2d(192),
            nn.ELU()
        )
        # 8 -> 39
        self.t2=nn.Sequential(
            nn.ConvTranspose2d(in_channels=192,out_channels=96,kernel_size=(1,8),stride=(1,5),padding=(0,2)),
            nn.BatchNorm2d(96),
            nn.ELU()
        )
        #39 -> 162
        self.t3=nn.Sequential(
            nn.ConvTranspose2d(in_channels=96,out_channels=48,kernel_size=(1,10),stride=(1,4),padding=0),
            nn.BatchNorm2d(48),
            nn.ELU()
        )
        #199 -> 1000
        self.t4=nn.Sequential(
            nn.ConvTranspose2d(in_channels=48,out_channels=24,kernel_size=(22, 1),stride=1,padding=0),
            nn.BatchNorm2d(24),
            nn.ELU()
        )
        #199 -> 1000
        self.t5=nn.Sequential(
            nn.ConvTranspose2d(in_channels=24,out_channels=1,kernel_size=(1,17),stride=(1,3),padding=0),
            nn.Tanh()
        )
    
    def forward(self,x,labels):
      x=x.view(-1,110)
      x = torch.mul(self.label_emb(labels), x)
      x=self.fc1(x)
      x=x.view(-1,384,1,1)
      x=self.t1(x)
      # print(x.shape)
      x=self.t2(x)
      # print(x.shape)
      x=self.t3(x)
      # print(x.shape)
      x=self.t4(x)
      # print(x.shape)
      x=self.t5(x)
      # print(x.shape)
      return x.squeeze() * 20 #output of generator

In [ ]:
import torch
from torch import nn
import torch.utils.data
import torchvision.datasets as data_set
import torchvision.transforms as transforms
from torch import optim
from torch.autograd import Variable
import numpy as np
import torchvision.utils as vutils

lr=0.0002
epochs=50
batch_size=128


def compute_acc(preds, labels):
	correct = 0
	preds_ = preds.data.max(1)[1]
	correct = preds_.eq(labels.data).cpu().sum()
	acc = float(correct) / float(len(labels.data)) * 100.0
	return acc

def weights_init(m):
		classname = m.__class__.__name__
		if classname.find('Conv') != -1:
			m.weight.data.normal_(0.0, 0.02)
		elif classname.find('BatchNorm') != -1:
			m.weight.data.normal_(1.0, 0.02)
			m.bias.data.fill_(0)


class EEGDataset(Dataset):
    """EEG dataset."""
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
          pass 
            # x = self.transform(x)
            # y = self.transform(y)
        return x, y
        
    def __len__(self):
        return len(self.subset)

init_dataset = TensorDataset(X_train_valid_tensor, y_train_valid_tensor) 


train_data = EEGDataset(
    init_dataset, transform=None)


dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)



gen=generator(110).to(device)
disc=discriminator().to(device)

gen.apply(weights_init)

optimD=optim.Adam(disc.parameters(),lr)
optimG=optim.Adam(gen.parameters(),lr * 2)

source_obj=nn.BCELoss()#source-loss

class_obj=nn.NLLLoss()#class-loss


for epoch in range(epochs):
  for i,data in enumerate(dataloader,0):
    '''
    At first we will train the discriminator
    '''
    #training with real data----
    optimD.zero_grad()

    image,label=data
    image,label=image.to(device),label.to(device)

    batch_size = len(label)

    real_label = torch.FloatTensor(batch_size).to(device)
    real_label.fill_(1)

    fake_label = torch.FloatTensor(batch_size).to(device)
    fake_label.fill_(0)

    source_,class_=disc(image)#we feed the real images into the discriminator
    #print(source_.size())
    source_error=source_obj(source_.squeeze(),real_label)#label for real images--1; for fake images--0
    class_error=class_obj(class_,label)
    error_real=source_error+class_error
    error_real.backward()
    optimD.step()


    accuracy=compute_acc(class_,label)#getting the current classification accuracy

    #training with fake data now----


    noise_ = np.random.normal(0, 1, (batch_size, 110))#generating noise by random sampling from a normal distribution

    label=np.random.randint(0,4,batch_size)#generating labels for the entire batch

    noise=((torch.from_numpy(noise_)).float())
    noise=noise.to(device)#converting to tensors in order to work with pytorch

    label=((torch.from_numpy(label)).long())
    label=label.to(device)#converting to tensors in order to work with pytorch

    noise_image=gen(noise, label)
    #print(noise_image.size())

    source_,class_=disc(noise_image.detach())#we will be using this tensor later on
    #print(source_.size())
    source_error=source_obj(source_.squeeze(),fake_label)#label for real images--1; for fake images--0
    class_error=class_obj(class_,label)
    error_fake=source_error+class_error
    error_fake.backward()
    optimD.step()


    '''
    Now we train the generator as we have finished updating weights of the discriminator
    '''

    gen.zero_grad()
    source_,class_=disc(noise_image)
    source_error=source_obj(source_.squeeze(),real_label)#The generator tries to pass its images as real---so we pass the images as real to the cost function
    class_error=class_obj(class_,label)
    error_gen=source_error+class_error
    error_gen.backward()
    optimG.step()
    iteration_now = epoch * len(dataloader) + i


    print("Epoch--[{} / {}], Loss_Discriminator--[{}], Loss_Generator--[{}],Accuracy--[{}]".format(epoch,epochs,error_fake,error_gen,accuracy))


torch.save(gen, "./gen")
torch.save(disc, "./disc")



Epoch--[0 / 50], Loss_Discriminator--[0.976993203163147], Loss_Generator--[0.13517755270004272],Accuracy--[21.875]
Epoch--[0 / 50], Loss_Discriminator--[1.2488107681274414], Loss_Generator--[0.026370376348495483],Accuracy--[23.4375]
Epoch--[0 / 50], Loss_Discriminator--[1.3743916749954224], Loss_Generator--[0.017680659890174866],Accuracy--[28.90625]
Epoch--[0 / 50], Loss_Discriminator--[1.3359071016311646], Loss_Generator--[0.11311545968055725],Accuracy--[22.65625]
Epoch--[0 / 50], Loss_Discriminator--[1.0778733491897583], Loss_Generator--[0.20117983222007751],Accuracy--[19.53125]
Epoch--[0 / 50], Loss_Discriminator--[0.9587048292160034], Loss_Generator--[0.27903056144714355],Accuracy--[27.34375]
Epoch--[0 / 50], Loss_Discriminator--[0.7829943299293518], Loss_Generator--[0.2828434705734253],Accuracy--[30.46875]
Epoch--[0 / 50], Loss_Discriminator--[0.6553894281387329], Loss_Generator--[0.2963603436946869],Accuracy--[25.0]
Epoch--[0 / 50], Loss_Discriminator--[0.6492322683334351], Loss_

In [ ]:
torch.cuda.empty_cache()
fake_sample_size = 2116
noise_ = np.random.normal(0, 1, (fake_sample_size, 110))#generating noise by random sampling from a normal distribution

label=np.random.randint(0,4,fake_sample_size)#generating labels for the entire batch

noise=((torch.from_numpy(noise_)).float())
noise=noise.to(device)#converting to tensors in order to work with pytorch

label=((torch.from_numpy(label)).long())
label=label.to(device)#converting to tensors in order to work with pytorch

# print(noise.shape)
# print(label.shape)
# model = torch.load("./disc").to(device)
# print(model)
# !pip install torchsummary
# from torchsummary import summary
# summary(model, [(1, 22,500)])

model = torch.load("./gen").to(device)

model.eval()

noise_image=model(noise, label)

print(noise_image.shape)
print(label.shape)

label += 769

fake_X_train_valid = noise_image.detach().cpu().numpy()
fake_y_train_valid = label.detach().cpu().numpy()
np.save("fake_X_train_valid.npy", fake_X_train_valid)
np.save("fake_y_train_valid.npy", fake_y_train_valid)

torch.Size([2116, 110])
torch.Size([2116])
discriminator(
  (conv11): Conv2d(1, 25, kernel_size=(1, 5), stride=(1, 1))
  (conv12): Conv2d(25, 25, kernel_size=(22, 1), stride=(1, 1))
  (batchnorm1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (dp): Dropout(p=0.5, inplace=False)
  (conv2): Conv2d(25, 50, kernel_size=(1, 5), stride=(1, 1))
  (batchnorm2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(50, 100, kernel_size=(1, 5), stride=(1, 1))
  (batchnorm3): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_class): Linear(in_features=5900, out_features=4, bias=True)
  (fc_source): Linear(in_features=5900, out_features=1, bias=True)
  (sig): Sigmoid()
  (soft): Softmax(dim=1)
)
----------------------------------------------------------------
        Layer (type)         